In [7]:
import numpy as np
import pandas as pd 

def validate_ifpr_bounds(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[0]):
            mu, v = matrix[i, j]
            if not (0 <= mu <= 1 and 0 <= v <= 1 and mu + v <= 1):
                return False
            if i == j and (mu != 0.5 or v != 0.5):  # Diagonal check
                mu, v = v, mu
    return True

def validate_input_ifpr(ifpr_matrix):
    """
    Validate input IFPR matrix format and bounds
    """
    if not isinstance(ifpr_matrix, np.ndarray):
        raise TypeError("Input must be a numpy array")
    if len(ifpr_matrix.shape) != 3 or ifpr_matrix.shape[0] != ifpr_matrix.shape[1]:
        raise ValueError("Input must be a square matrix of shape (n, n, 2)")
    if not validate_ifpr_bounds(ifpr_matrix):
        raise ValueError("Input IFPR matrix contains invalid entries")

def construct_perfect_multiplicative_consistent_ifpr(ifpr_matrix):
    n = ifpr_matrix.shape[0]
    R_bar = np.zeros_like(ifpr_matrix)
    
    for i in range(n):
        for k in range(n):
            if i == k:
                R_bar[i, k] = (0.5, 0.5)
            elif k > i:
                mu_product = np.prod([ifpr_matrix[t, t+1][0] for t in range(i, k)])
                mu_complement_product = np.prod([1 - ifpr_matrix[t, t+1][0] for t in range(i, k)])
                mu_ik = mu_product / (mu_product + mu_complement_product)
                
                v_product = np.prod([ifpr_matrix[t, t+1][1] for t in range(i, k)])
                v_complement_product = np.prod([1 - ifpr_matrix[t, t+1][1] for t in range(i, k)])
                v_ik = v_product / (v_product + v_complement_product)
                
                if mu_ik + v_ik > 1:
                    excess = mu_ik + v_ik - 1
                    v_ik -= excess  # Adjust non-membership only
                
                R_bar[i, k] = (mu_ik, v_ik)
            else:
                R_bar[i, k] = (R_bar[k, i][1], R_bar[k, i][0])
    
    return R_bar

R_bar = construct_perfect_multiplicative_consistent_ifpr(R)
def calculate_distance(R_bar, R_p):
    """
    Calculates the distance between two IFPR matrices.
    """
    validate_input_ifpr(R_bar)
    validate_input_ifpr(R_p)
    
    if R_bar.shape != R_p.shape:
        raise ValueError("Matrices must have the same shape")
    
    n = R_bar.shape[0]
    distance_sum = 0
    
    for i in range(n):
        for k in range(n):
            distance_sum += abs(R_bar[i, k][0] - R_p[i, k][0]) + abs(R_bar[i, k][1] - R_p[i, k][1])
    
    return distance_sum / (2 * (n - 1) * (n - 2))

def fuse_ifpr_matrices(R_p, R_bar, sigma):
    """
    Fuses two IFPR matrices using the given sigma parameter.
    """
    validate_input_ifpr(R_p)
    validate_input_ifpr(R_bar)
    
    if not 0 <= sigma <= 1:
        raise ValueError("Sigma must be between 0 and 1")
    
    n = R_p.shape[0]
    R_tilde = np.zeros_like(R_p)
    
    for i in range(n):
        for k in range(n):
            if i == k:
                R_tilde[i, k] = (0.5, 0.5)
            else:
                mu_p, v_p = R_p[i, k]
                mu_bar, v_bar = R_bar[i, k]
                
                mu_p_safe = max(mu_p, 1e-6)
                v_p_safe = max(v_p, 1e-6)
                mu_bar_safe = max(mu_bar, 1e-10)
                v_bar_safe = max(v_bar, 1e-10)
                
                mu_tilde = ((mu_p_safe ** (1 - sigma)) * (mu_bar_safe ** sigma)) / \
                           (((mu_p_safe ** (1 - sigma)) * (mu_bar_safe ** sigma)) +
                            ((1 - mu_p_safe) ** (1 - sigma)) * ((1 - mu_bar_safe) ** sigma))
                v_tilde = ((v_p_safe ** (1 - sigma)) * (v_bar_safe ** sigma)) / \
                          (((v_p_safe ** (1 - sigma)) * (v_bar_safe ** sigma)) +
                           ((1 - v_p_safe) ** (1 - sigma)) * ((1 - v_bar_safe) ** sigma))
                
                if mu_tilde + v_tilde > 1:
                    excess = mu_tilde + v_tilde - 1
                    v_tilde -= excess  # Adjust non-membership only
                
                R_tilde[i, k] = (mu_tilde, v_tilde)
    
    return R_tilde

def transform_ifpr_to_ivpr(ifpr_matrix):
    """
    Transforms an IFPR matrix into an Interval-Valued Preference Relation (IVPR).
    """
    n = ifpr_matrix.shape[0]
    ivpr_matrix = np.zeros((n, n, 2))
    
    for i in range(n):
        for k in range(n):
            mu_ik, v_ik = ifpr_matrix[i, k]
            mu_ik = max(0, min(mu_ik, 1))
            v_ik = max(0, min(v_ik, 1))
            
            upper_bound = min(1, mu_ik + (1 - v_ik))
            ivpr_matrix[i, k] = [mu_ik, upper_bound - mu_ik]
    
    return ivpr_matrix

def calculate_priority_vector(ivpr_matrix):
    """
    Calculates the priority vector while ensuring μ + v + π = 1 constraint.
    """
    n = ivpr_matrix.shape[0]
    
    total_mu = np.sum(ivpr_matrix[:, :, 0])
    total_v_complement = np.sum(ivpr_matrix[:, :, 1])
    
    if total_mu == 0 or total_v_complement == 0:
        raise ValueError("Division by zero encountered. Check the IFPR matrix for invalid values.")
    
    priority_vector = []
    for i in range(n):
        mu_i = np.sum(ivpr_matrix[i, :, 0]) / total_mu
        v_i = 1 - (np.sum(ivpr_matrix[i, :, 1]) / total_v_complement)

        # Calculate hesitancy (π)
        pi_i = max(0, 1 - mu_i - v_i)
        
        # Ensure μ + v + π = 1
        if mu_i + v_i + pi_i != 1:
            # Adjust v_i to ensure μ + v + π = 1
            v_i = 1 - mu_i - pi_i
        
        priority_vector.append((mu_i, v_i, pi_i))
    
    return priority_vector
    
def calculate_crisp_weights(priority_vector):
    """
    Computes crisp weights ensuring μ + (1 - μ - v) / 2 is valid, and includes hesitancy.
    """
    crisp_weights = []
    for mu, v, pi in priority_vector:
        # Calculate crisp weight including hesitancy
        crisp_weight = mu + (pi / 2)
        crisp_weights.append(crisp_weight)
    
    # Normalize weights to sum to 1
    total_weight = sum(crisp_weights)
    normalized_weights = [w / total_weight for w in crisp_weights]
    
    return normalized_weights

def calculate_consistency_ratio(original_ifpr, repaired_ifpr):
    initial_distance = calculate_distance(original_ifpr, repaired_ifpr)
    perfect_distance = calculate_distance(repaired_ifpr, construct_perfect_multiplicative_consistent_ifpr(repaired_ifpr))
    
    if initial_distance < 1e-6:  # Avoid division by very small numbers
        return 0
    return perfect_distance / initial_distance

def check_and_repair_ifpr(ifpr_matrix, tau=0.1, sigma=0.5, max_iter=100):
    """
    Checks and repairs the IFPR matrix for consistency and calculates weights.
    
    Returns:
    - tuple: (repaired_matrix, crisp_weights, consistency_ratio, iterations, priority_vector)
    """
    validate_input_ifpr(ifpr_matrix)
    
    if not (0 < tau < 1):
        raise ValueError("Tau must be between 0 and 1")
    
    n = ifpr_matrix.shape[0]
    p = 1
    R_p = ifpr_matrix.copy()
    
    while p <= max_iter:
        R_bar = construct_perfect_multiplicative_consistent_ifpr(R_p)
        d_R = calculate_distance(R_bar, R_p)
        
        if d_R < tau:
            print(f"Matrix is consistent after {p} iterations")
            break
        
        R_tilde = fuse_ifpr_matrices(R_p, R_bar, sigma)
        R_p = R_tilde
        p += 1
    
    if p > max_iter:
        print(f"Reached maximum iterations ({max_iter}). Returning last repaired matrix.")
    
    # Transform IFPR to IVPR and calculate weights
    ivpr_matrix = transform_ifpr_to_ivpr(R_p)
    priority_vector = calculate_priority_vector(ivpr_matrix)
    crisp_weights = calculate_crisp_weights(priority_vector)
    
    # Calculate consistency ratio
    consistency_ratio = calculate_consistency_ratio(ifpr_matrix, R_p)
    
    return R_p, crisp_weights, consistency_ratio, p, priority_vector

# Example usage
if __name__ == "__main__":
    # Example IFPR matrix
    '''
    ifpr_matrix = np.array([
        [[0.5, 0.1], [0.2, 0.6], [0.1, 0.8]],
        [[0.6, 0.2], [0.5, 0.2], [0.3, 0.6]],
        [[0.8, 0.1], [0.6, 0.3], [0.5, 0.3]]
    ])
    '''
    '''
    # balanced preference
    ifpr_matrix = np.array([
    [[0.5, 0.3], [0.4, 0.4], [0.4, 0.4]],
    [[0.4, 0.4], [0.5, 0.3], [0.4, 0.4]],
    [[0.4, 0.4], [0.4, 0.4], [0.5, 0.3]]
    ])
    '''

    '''
    # cost focused
    ifpr_matrix  = np.array([
    [[0.5, 0.3], [0.7, 0.2], [0.6, 0.3]],
    [[0.2, 0.7], [0.5, 0.3], [0.3, 0.5]],
    [[0.3, 0.6], [0.5, 0.3], [0.5, 0.3]]
    ])
    '''
    '''
    # Value-focused
    ifpr_matrix = np.array([
    [[0.5, 0.3], [0.3, 0.6], [0.4, 0.5]],
    [[0.6, 0.3], [0.5, 0.3], [0.7, 0.2]],
    [[0.5, 0.4], [0.2, 0.7], [0.5, 0.3]]
    ])
    '''

    '''
    # importance-focused
    ifpr_matrix = np.array([
    [[0.5, 0.3], [0.4, 0.5], [0.2, 0.7]],
    [[0.5, 0.4], [0.5, 0.3], [0.3, 0.6]],
    [[0.7, 0.2], [0.6, 0.3], [0.5, 0.3]]
    ])
    '''
    '''        
    # high uncertainty
    ifpr_matrix = np.array([
    [[0.5, 0.2], [0.4, 0.3], [0.3, 0.2]],
    [[0.3, 0.4], [0.5, 0.2], [0.4, 0.3]],
    [[0.2, 0.3], [0.3, 0.4], [0.5, 0.2]]
    ])
    '''
    # case study RESEARCH PAPER
    ifpr_matrix = np.array([
    [[0.5, 0.5], [0.6, 0.2], [0.6, 0.2], [0.65, 0.25], [0.65, 0.25]],
    [[0.2, 0.6], [0.5, 0.5], [0.6, 0.2], [0.6, 0.2], [0.6, 0.2]],
    [[0.2, 0.6], [0.2, 0.6], [0.5, 0.5], [0.6, 0.2], [0.6, 0.2]],
    [[0.25, 0.65], [0.25, 0.65], [0.2, 0.6], [0.5, 0.5], [0.6, 0.2]],
    [[0.25, 0.65], [0.25, 0.65], [0.2, 0.6], [0.2, 0.6], [0.5, 0.5]]
    ])
    # Run consistency check and repair
    repaired_matrix, weights, consistency_ratio, iterations, priority_vector = check_and_repair_ifpr(ifpr_matrix)
    
    print("\nRepaired IFPR Matrix:")
    print(repaired_matrix)
    
    print("\nPriority Vector (Weights for Each Criteria):")
    for i, weight in enumerate(priority_vector):
        print(f"Criterion {i+1}: μ = {weight[0]:.4f}, v = {weight[1]:.4f}, π = {weight[2]:.4f}")
    
    print("\nCrisp Weights:")
    for i, weight in enumerate(weights):
        print(f"Criterion {i+1}: {weight:.4f}")
    
    print("\nConsistency Ratio:", consistency_ratio)
    print("Number of iterations:", iterations)

Matrix is consistent after 2 iterations

Repaired IFPR Matrix:
[[[0.5        0.5       ]
  [0.6        0.2       ]
  [0.64752955 0.11111111]
  [0.71457675 0.06731103]
  [0.75407214 0.03482766]]

 [[0.2        0.6       ]
  [0.5        0.5       ]
  [0.6        0.2       ]
  [0.64752955 0.11111111]
  [0.69230769 0.05882353]]

 [[0.11111111 0.64752955]
  [0.2        0.6       ]
  [0.5        0.5       ]
  [0.6        0.2       ]
  [0.64752955 0.11111111]]

 [[0.06731103 0.71457675]
  [0.12613198 0.67150166]
  [0.2        0.6       ]
  [0.5        0.5       ]
  [0.6        0.2       ]]

 [[0.03482766 0.75407214]
  [0.06731103 0.71457675]
  [0.11111111 0.64752955]
  [0.2        0.6       ]
  [0.5        0.5       ]]]

Priority Vector (Weights for Each Criteria):
Criterion 1: μ = 0.3116, v = 0.6884, π = 0.0000
Criterion 2: μ = 0.2558, v = 0.7442, π = 0.0000
Criterion 3: μ = 0.1995, v = 0.7878, π = 0.0128
Criterion 4: μ = 0.1447, v = 0.7974, π = 0.0579
Criterion 5: μ = 0.0885, v = 0.8112, π 

<h3> 
    
the above method u and v values have hesitancy removed for some reason, mostly because of how matrix is being converted and repaired into Multiplicative Consistency Matrix. </h3>

<h3> To fix this start from this case study provided in research paper 1 of IFAHP , implement multiplicative consistency as per the paper 1 and i got the values a little bit off </h3>
<br>the output should look like this </br> 

### Perfect Multiplicative Consistent IFPR Matrix:

| Row/Col | 1        | 2        | 3        | 4        | 5        |
|---------|----------|----------|----------|----------|----------|
| **1**   | (0.5, 0.5) | (0.6, 0.2) | (0.6923, 0.0588) | (0.7146, 0.0673) | (0.8069, 0.0204) |
| **2**   | (0.2, 0.6) | (0.5, 0.5) | (0.6, 0.2) | (0.6923, 0.0588) | (0.7146, 0.0673) |
| **3**   | (0.0588, 0.6923) | (0.2, 0.6) | (0.5, 0.5) | (0.6, 0.2) | (0.6923, 0.0588) |
| **4**   | (0.0673, 0.7146) | (0.0588, 0.6923) | (0.2, 0.6) | (0.5, 0.5) | (0.6, 0.2) |
| **5**   | (0.0204, 0.8069) | (0.0673, 0.7146) | (0.0588, 0.6923) | (0.2, 0.6) | (0.5, 0.5) |
.5        0.5       ]]] </h3>

In [211]:
import numpy as np

# Initial IFPR matrix for criteria with respect to the overall objective
R = np.array([
    [[0.5, 0.5], [0.6, 0.2], [0.6, 0.2], [0.65, 0.25], [0.65, 0.25]],
    [[0.2, 0.6], [0.5, 0.5], [0.6, 0.2], [0.65, 0.25], [0.65, 0.25]],
    [[0.4, 0.2], [0.4, 0.2], [0.5, 0.5], [0.6, 0.2], [0.6, 0.2]],
    [[0.35, 0.65], [0.35, 0.65], [0.4, 0.2], [0.5, 0.5], [0.6, 0.2]],
    [[0.35, 0.65], [0.35, 0.65], [0.4, 0.2], [0.4, 0.2], [0.5, 0.5]]
])

In [207]:
R = np.array([
    [[0.5, 0.5], [0.2, 0.6], [0.3, 0.4], [0.6, 0.2]],
    [[0.6, 0.2], [0.5, 0.5], [0.5, 0.4], [0.6, 0.4]],
    [[0.4, 0.3], [0.4, 0.5], [0.5, 0.5], [0.3, 0.2]],
    [[0.2, 0.6], [0.4, 0.6], [0.2, 0.3], [0.5, 0.5]]
])

In [ ]:
R = 

<h3> Method 1 incorporating the formula from research paper 1 of IFAHP </h3>

In [225]:
def construct_perfect_multiplicative_consistent_ifpr(R):
    """
    Constructs the Perfect Multiplicative Consistent IFPR (R_bar) from the original IFPR R.
    
    For i < k, if we define the chain over t = i+1, ..., k-1, then:
    
       μ̃(i,k) = { [∏ₜ ( μ(i,t) * μ(t,k) )]^(exponent) } /
                  { [∏ₜ ( μ(i,t) * μ(t,k) )]^(exponent) + [∏ₜ ( (1-μ(i,t)) * (1-μ(t,k)) )]^(exponent) }
    
       ν̃(i,k) is defined similarly using ν.
    
    The exponent is chosen as follows:
      - If chain length = (k - i - 1) equals 1, exponent = 1.
      - If chain length > 1, exponent = 0.5 (i.e. take square root).
    
    For adjacent indices (k = i+1), the original values are kept.
    The lower-triangular part is filled by reciprocity:
      R_bar[k,i] = (ν̃(i,k), μ̃(i,k))
    """
    n = R.shape[0]
    R_bar = np.zeros((n, n, 2), dtype=float)

    # 1. Set diagonal entries to (0.5, 0.5)
    for i in range(n):
        R_bar[i, i] = [0.5, 0.5]

    # 2. Compute upper-triangular entries
    for i in range(n):
        for k in range(i+1, n):
            if k == i + 1:
                # For adjacent indices, simply use the original value.
                R_bar[i, k] = R[i, k]
            else:
                exponent = 1.0 / (k - i - 1)
                # Compute chain-products for membership
                prod_mu = 1.0
                prod_1mu = 1.0
                for t in range(i+1, k):
                    prod_mu   *= (R[i, t, 0] * R[t, k, 0])
                    prod_1mu  *= ((1 - R[i, t, 0]) * (1 - R[t, k, 0]))
                left_mu  = prod_mu ** exponent
                right_mu = prod_1mu ** exponent
                denom_mu = left_mu + right_mu
                new_mu = left_mu / denom_mu if denom_mu != 0 else 0.5

                # Compute chain-products for non-membership
                prod_nu = 1.0
                prod_1nu = 1.0
                for t in range(i+1, k):
                    prod_nu   *= (R[i, t, 1] * R[t, k, 1])
                    prod_1nu  *= ((1 - R[i, t, 1]) * (1 - R[t, k, 1]))
                left_nu  = prod_nu ** exponent
                right_nu = prod_1nu ** exponent
                denom_nu = left_nu + right_nu
                new_nu = left_nu / denom_nu if denom_nu != 0 else 0.5

                R_bar[i, k] = [new_mu, new_nu]

    # 3. Fill the lower-triangular part using reciprocity:
    # If R_bar[i,k] = (μ, ν), then set R_bar[k,i] = (ν, μ).
    for i in range(n):
        for k in range(i+1, n):
            mu_val, nu_val = R_bar[i, k]
            R_bar[k, i] = [nu_val, mu_val]

    return R_bar

In [227]:
R_bar = construct_perfect_multiplicative_consistent_ifpr(R)
# Print results with better formatting
np.set_printoptions(precision=4, suppress=True)
print("Original R (upper triangle):")
print(R[..., 0], "\n", R[..., 1], "\n")
print("Perfect Multiplicative Consistent IFPR (R_bar):")
for i in range(5):
    row_str = []
    for k in range(5):
        mu_val = R_bar[i, k, 0]
        nu_val = R_bar[i, k, 1]
        row_str.append(f"({mu_val:.4f}, {nu_val:.4f})")
    print(f"Row {i+1}:", "\t".join(row_str))

Original R (upper triangle):
[[0.5  0.6  0.6  0.65 0.65]
 [0.2  0.5  0.6  0.65 0.65]
 [0.4  0.4  0.5  0.6  0.6 ]
 [0.35 0.35 0.4  0.5  0.6 ]
 [0.35 0.35 0.4  0.4  0.5 ]] 
 [[0.5  0.2  0.2  0.25 0.25]
 [0.6  0.5  0.2  0.25 0.25]
 [0.2  0.2  0.5  0.2  0.2 ]
 [0.65 0.65 0.2  0.5  0.2 ]
 [0.65 0.65 0.2  0.2  0.5 ]] 

Perfect Multiplicative Consistent IFPR (R_bar):
Row 1: (0.5000, 0.5000)	(0.6000, 0.2000)	(0.6923, 0.0588)	(0.7146, 0.0673)	(0.7218, 0.0704)
Row 2: (0.2000, 0.6000)	(0.5000, 0.5000)	(0.6000, 0.2000)	(0.6923, 0.0588)	(0.7146, 0.0673)
Row 3: (0.0588, 0.6923)	(0.2000, 0.6000)	(0.5000, 0.5000)	(0.6000, 0.2000)	(0.6923, 0.0588)
Row 4: (0.0673, 0.7146)	(0.0588, 0.6923)	(0.2000, 0.6000)	(0.5000, 0.5000)	(0.6000, 0.2000)
Row 5: (0.0704, 0.7218)	(0.0673, 0.7146)	(0.0588, 0.6923)	(0.2000, 0.6000)	(0.5000, 0.5000)


In [257]:
def check_consistency(R, R_bar, tau=0.1):
    """
    Computes the distance d between the original IFPR matrix R and 
    the perfectly multiplicative consistent IFPR matrix R_bar using:
    
      d(R, R_bar) = (1 / ((n-1) * (n-2))) * sum_{i=1}^{n} sum_{k=i+1}^{n} 
                     [|mu_bar(i,k) - mu(i,k)| + |nu_bar(i,k) - nu(i,k)| + |pi_bar(i,k) - pi(i,k)|],
                     
    where for each entry,
      pi(i,k) = 1 - mu(i,k) - nu(i,k) and
      pi_bar(i,k) = 1 - mu_bar(i,k) - nu_bar(i,k).
    
    Parameters:
      R      : Original IFPR matrix (n x n x 2 numpy array)
      R_bar  : Consistent IFPR matrix (n x n x 2 numpy array)
      tau    : Threshold for acceptable consistency (default: 0.1)
    
    Returns:
      (bool) True if d < tau, else False.
    """
    n = R.shape[0]
    total_diff = 0.0
    
    # Loop over the upper triangular indices (i < k)
    for i in range(n):
        for k in range(i+1, n):
            mu = R[i, k, 0]
            nu = R[i, k, 1]
            pi = 1 - mu - nu
            
            mu_bar = R_bar[i, k, 0]
            nu_bar = R_bar[i, k, 1]
            pi_bar = 1 - mu_bar - nu_bar
            
            diff = abs(mu_bar - mu) + abs(nu_bar - nu) + abs(pi_bar - pi)
            total_diff += diff
    
    # Use normalization factor as in the paper: (n-1)*(n-2) or 2*(n-1)(n-2) idk man
    denominator = 2* (n - 1) * (n - 2)
    d = total_diff / denominator
    
    print("Distance d(R, R_bar) =", d)
    return d < tau

In [259]:
R_bar = construct_perfect_multiplicative_consistent_ifpr(R)
    
# Check consistency:
consistent = check_consistency(R, R_bar, tau=0.1)
if consistent:
    print("The IFPR is consistent (d < 0.1).")
else:
    print("The IFPR is NOT consistent (d >= 0.1).")

Distance d(R, R_bar) = 0.08487691917587832
The IFPR is consistent (d < 0.1).


In [18]:
def fuse_ifpr_matrices(R_p, R_star, sigma=0.5):
    """
    Eqs. (23) and (24):
      mu^(p+1)_{i,k} = [ (mu^(p)_{i,k})^(1-sigma) * (mu^*_{i,k})^sigma ] /
                       [ ... + ... ],
      v^(p+1)_{i,k}  = similarly for non-membership.

    R_p, R_star: shape (n,n,2) => (mu, nu).
    sigma in [0,1].
    Returns R_{p+1} of same shape.
    """
    n = R_p.shape[0]
    R_next = np.zeros_like(R_p)

    for i in range(n):
        for k in range(n):
            mu_p, v_p = R_p[i, k]
            mu_s, v_s = R_star[i, k]

            # Avoid zero denominators
            eps = 1e-12
            mu_p_safe = max(mu_p, eps)
            mu_s_safe = max(mu_s, eps)
            v_p_safe  = max(v_p, eps)
            v_s_safe  = max(v_s, eps)

            # membership
            numerator_mu = (mu_p_safe**(1-sigma)) * (mu_s_safe**sigma)
            denominator_mu = (
                (mu_p_safe**(1-sigma))*(mu_s_safe**sigma)
                + ((1 - mu_p_safe)**(1-sigma))*((1 - mu_s_safe)**sigma)
            )
            mu_new = numerator_mu / max(denominator_mu, eps)

            # non-membership
            numerator_v = (v_p_safe**(1-sigma)) * (v_s_safe**sigma)
            denominator_v = (
                (v_p_safe**(1-sigma))*(v_s_safe**sigma)
                + ((1 - v_p_safe)**(1-sigma))*((1 - v_s_safe)**sigma)
            )
            v_new = numerator_v / max(denominator_v, eps)

            # If mu_new + v_new > 1 => subtract from v_new (or distribute)
            sum_mv = mu_new + v_new
            if sum_mv > 1.0:
                excess = sum_mv - 1.0
                v_new -= excess

            R_next[i, k] = [mu_new, v_new]

    return R_next


In [20]:
def algorithm_II_improve_inconsistency(R_init, tau=0.1, sigma=0.5, max_iter=50):
    """
    Algorithm II:
      Step 1: p=1
      Step 2: R_star <- construct_perfect_multiplicative_consistent_ifpr(R_init)
      Step 3: d_val = d(R_star, R^(p))
                if d_val < tau => done => return R^(p)
      Step 4: else R^(p+1) = fuse_ifpr_matrices(R^(p), R_star, sigma)
      Step 5: p++ => go to step 3
      up to max_iter times.

    Returns:
      R_star: the perfect matrix from Algorithm I
      R_fused: the final R^(p+1)
      p_used: how many iterations used
    """
    # Step 2: Build perfect matrix from Algorithm I
    R_star = construct_perfect_multiplicative_consistent_ifpr(R_init)

    # Start iteration
    R_current = R_init.copy()  # R^(1)
    for p in range(1, max_iter+1):
        d_val = distance_ifpr(R_star, R_current)  # eq. (22)
        if d_val < tau:
            return R_star, R_current, p

        # If not consistent, fuse => eq. (23),(24)
        R_next = fuse_ifpr_matrices(R_current, R_star, sigma)
        R_current = R_next

    # If we exit the loop, we didn't reach < tau
    return R_star, R_current, max_iter


In [26]:
if __name__ == "__main__":
    # Suppose we define an initial IFPR matrix R_init
    # shape (n,n,2): (mu_{i,j}, nu_{i,j}).
    # We'll just mock something for demonstration:
    R_init = R_bar
    # Run Algorithm II
    tau_val = 0.1
    sigma_val = 0.5
    R_star, R_fused, iterations_used = algorithm_II_improve_inconsistency(
        R_init, tau=tau_val, sigma=sigma_val, max_iter=50
    )

    print("\n--- Algorithm II Results ---")
    print(f"Used {iterations_used} iteration(s)")
    print("Perfect Matrix R_star (Algorithm I):", R_star)
    print("Final Fused Matrix R_fused:", R_fused)
    dist_final = distance_ifpr(R_star, R_fused)
    print(f"Final distance = {dist_final:.4f}")


--- Algorithm II Results ---
Used 1 iteration(s)
Perfect Matrix R_star (Algorithm I): [[[0.5        0.5       ]
  [0.6        0.2       ]
  [0.69230769 0.05882353]
  [0.744605   0.02674   ]
  [0.83088789 0.0094285 ]]

 [[0.2        0.6       ]
  [0.5        0.5       ]
  [0.6        0.2       ]
  [0.69230769 0.05882353]
  [0.744605   0.02674   ]]

 [[0.05882353 0.69230769]
  [0.2        0.6       ]
  [0.5        0.5       ]
  [0.6        0.2       ]
  [0.69230769 0.05882353]]

 [[0.02674    0.744605  ]
  [0.05882353 0.69230769]
  [0.2        0.6       ]
  [0.5        0.5       ]
  [0.6        0.2       ]]

 [[0.0094285  0.83088789]
  [0.02674    0.744605  ]
  [0.05882353 0.69230769]
  [0.2        0.6       ]
  [0.5        0.5       ]]]
Final Fused Matrix R_fused: [[[0.5        0.5       ]
  [0.6        0.2       ]
  [0.69230769 0.05882353]
  [0.71435714 0.067     ]
  [0.80729381 0.04080739]]

 [[0.2        0.6       ]
  [0.5        0.5       ]
  [0.6        0.2       ]
  [0.69230769 0